In [ ]:
%matplotlib qt

import hyperspy.api as hs
from generalizedplanck.components import (GeneralizedPlanck, Reflectance, 
                                          IdealSqrtAbsorption, UrbachTail)
from generalizedplanck.utils import get_nk, fit_signal
from generalizedplanck.data import gaas_cl
import matplotlib.pyplot as plt
import numpy as np

Create a fake GaAs CL signal

In [ ]:
s = gaas_cl()

Creating a reflectance object for the Generalised Planck law

In [ ]:
n1 = get_nk(n=1)
n2 = get_nk(shelf='main', book='GaAs', page='Papatryfonos')
gaas_r = Reflectance(theta=0, n1=n1, n2=n2, pol=None) 

Create an ideal sqrt absorptio coefficient plus an Urbach tail

In [ ]:
gaas_ideal_abs = IdealSqrtAbsorption(Eg=1.42, E0=1.6, a0=14800, include_tail=False, g=0.015,)
gaas_tail = UrbachTail(g=0.015)

Here you can define your Generalised Planck component

In [ ]:
gaas_genp = GeneralizedPlanck(Eg=1.42, g=0.01, p=0.8, T=300, d=150, 
                              Efv=0, Efc=0.1,
                              analytical=True,
                              reflectance=gaas_r,
                              ideal_abs_coeff=gaas_ideal_abs,
                              tail=gaas_tail)

You can try to plot it

In [ ]:
en = np.linspace(1.2, 1.8, 1000)
y = gaas_genp.function(en)
plt.plot(en,y)

If you want to fit some data you can use the `fit_signal` function. If `gui` is `True` you can fit by hand your data. If it is `False` it fits directly the data.

In [ ]:
component_list = [{'id_name' : 'GeneralizedPlanck',
                   'name': 'peak@1.39eV',
                   'kwargs' : {'analytical':True,
                               'reflectance':gaas_r,
                               'ideal_abs_coeff':gaas_ideal_abs,
                               'tail':gaas_tail},
                   'Eg':[1.39,1.39,1.42, True],
                   'g':[0.01, 0.001, 0.05, True],
                   'p':[False, 0.89],
                   'd':[2000, False],
                   'T':[295, False],
                   'Efv':[0, -0.1, 0.1, True],
                   'Efc':[-0.5, False]}
                 ]

In [ ]:
m = fit_signal(s, components=component_list, gui=True)